### LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import re

### TOPIC: Ecological Footprint and People Happiness

Q1: Do happiest countries have the best Ecological Footprint and Biocapacity?

Q2: How are the the components of the "Happiness Index" correlated with different components of the ecological footprint of a country?

Q3: Is it possible to model the level of happiness of a country based the components of its ecological footprint?

### FUNCTIONS

In [2]:
def general_check(df):
    """
    returns a dictionary of columns names, shape of the df, dict with columns as keys and
    number of nulls/columns
    
    Input: DataFrame
    Output:dictionary with nested dict and lists
    """
    general = {"COLNAMES":[c for c in df.columns],"SHAPE":tuple(df.shape), "TYPE":\
               {k:v for k,v in zip([c for c in df.columns],[df[c].dtype for\
                                                            c in [c for c in df.columns]])},
               "NULLS":{k:v for k,v in zip([c for c in df.columns],[df[c].isna().sum() for\
                                                            c in [c for c in df.columns]])}}
    return general


def col_uniques(df):
    """
    returns a dictionary of names of columns as keys and list of unique values of that column
    as values
    
    Input: DataFrame
    Output:dictionary
    """
    uniques = {k:v for k,v in zip([c.upper() for c in df.columns],[df[c].unique() for\
                                                           c in [c for c in df.columns]])}
    return uniques

### IMPORT DATASETS

#### 1) NATIONAL  FOOTPRINT ACCOUNTS

In [3]:
nfa = pd.read_csv("NFA 2018.csv")
nfa.head(10)

,country,ISO alpha-3 code,UN_region,UN_subregion,year,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,Percapita GDP (2010 USD),population
0,Armenia,ARM,Asia,Western Asia,1992,BiocapPerCap,1.611286e-01,0.135023,0.083836,0.013718,0.033669,0.000000e+00,4.273741e-01,949.033,3449000
1,Armenia,ARM,Asia,Western Asia,1992,BiocapTotGHA,5.558130e+05,465763.337400,289190.662300,47320.224590,116139.598200,0.000000e+00,1.474227e+06,949.033,3449000
2,Armenia,ARM,Asia,Western Asia,1992,EFConsPerCap,3.909225e-01,0.189137,0.000001,0.004138,0.033669,1.112225e+00,1.730092e+00,949.033,3449000
3,Armenia,ARM,Asia,Western Asia,1992,EFConsTotGHA,1.348487e+06,652429.066600,4.327841,14272.803690,116139.598200,3.836620e+06,5.967954e+06,949.033,3449000
4,Armenia,ARM,Asia,Western Asia,1992,EFExportsPerCap,1.124910e-03,0.002283,0.000000,0.000438,0.000000,4.819043e-02,5.203676e-02,949.033,3449000
5,Armenia,ARM,Asia,Western Asia,1992,EFExportsTotGHA,3.880378e+03,7875.331688,0.000000,1512.195296,0.000000,1.662329e+05,1.795008e+05,949.033,3449000
6,Armenia,ARM,Asia,Western Asia,1992,EFImportsPerCap,2.309189e-01,0.056397,0.000001,0.003312,0.000000,8.791121e-02,3.785406e-01,949.033,3449000
7,Armenia,ARM,Asia,Western Asia,1992,EFImportsTotGHA,7.965547e+05,194541.060900,4.327841,11426.058040,0.000000,3.032497e+05,1.305776e+06,949.033,3449000
8,Armenia,ARM,Asia,Western Asia,1992,EFProdPerCap,1.611286e-01,0.135023,0.000000,0.001264,0.033669,1.072504e+00,1.403588e+00,949.033,3449000
9,Armenia,ARM,Asia,Western Asia,1992,EFProdTotGHA,5.558130e+05,465763.337400,0.000000,4358.940944,116139.598200,3.699604e+06,4.841678e+06,949.033,3449000


<b>DESCRIPTION OF COLUMNS:</b>

- <b>countryCountry:</b> name

- <b>ISO alpha-3:</b> code

- <b>UN_region:</b> Region of the country

- <b>UN_subregion:</b> Subregion of the country

- <b>year:</b> Year in which values are calculated

- <b>record:</b> Description of reported global hectares

- <b>crop_land:</b> Global hectares of crop land (used for crops & crop-derived products)  available or demanded.</b>Biocapacity = Footprint of production because all biocapacity is used for production.</b>

- <b>grazing_land:</b> Global hectares of grazing land (used for meat, dairy, leather, etc.) available or demanded. Includes global hectares used for grazing, but not crop land used to produce feed for animals.

- <b>forest_land:</b> Global hectares of forest land available (for sequestration and timber, pulp, or timber products) or demanded (for timber, pulp, or timber products).

- <b>fishing_ground:</b> Global hectares of marine and inland fishing grounds (used for fish & fish products) available or demanded.

- <b>built_up_land:</b> Global hectares of built-up land (land cover of human infrastructure) available or demanded. Not traded. <b>Biocapacity equals Footprint of Production because 100% of biocapacity covered counts as that demanded by infrastructure coverage.</b>

- <b>carbon:</b> Global hectares of world-average forest required to sequester carbon emissions.

- <b>total:</b> The sum of all land types for this country, year, and record

- <b>Percapita GDP (2010 USD):</b> Per capita GDP in constant 2010 USD (source: World Bank)

- <b>population:</b> population rounded to thousands (FAO estimate)

<b>SUMMARY:</b>

In this data set we show results for 196 countries (including "World") for data years 1961 through 2014, the most recent year with complete data. For each country and year, we report Ecological Footprint of Consumption, also just called Ecological Footprint, and Biocapacity ("Biocap...","EFCons...") as well as Ecological Footprint of Production ("EFProd...") and trade ("EFImports...", "EFExports...") in both total global hectares and global hectares per capita ("...TotGHA","...PerCap"). For more details on these types, see below.

The columns "crop_land" through "carbon" are the number of global hectares of this landtype either required to support consumption/production (Ecological Footprint of Consumption, Production), or that are supported by biological productivity (Biocapacity). The "total" column is the sum of these values.

You will notice that Carbon is 0 in biocapacity, which is because the biocapacity of carbon sequestration is contained in forest. You will also notice that for some countries and years, data is either entirely not available or only available in the "total" column. This is because we have intentionally filtered available data to provide only country-years whose footprint and biocapacity results (either in detail or at least total) seem trustworthy.

<b>Method details and units:</b>

The only unit of measure in this dataset is the global hectare. This is a unit of land normalized by biological productivity across landtype, country, and year. We use this unit in lieu of physical area because of the need to compare usage of different regions, times, and land types on the same footing. This lies at the heart of footprinting methodology.

Imagine, for example, a physical hectare of rich crop land and an equal area of arid grassland. Though the same area, they comprise very different capacities for biological regrowth and absorption, and saying that you only require a hectare of desert is very different than saying you require a hectare of rich crop land. When we normalize into global hectares, it makes sense to compare equal amounts of Footprint and Biocapacity between countries, landtype, and time, even if we have to sacrifice an easy physical quantification. Requiring a global hectare to support your consumption means the same thing coming from anyone!

The Ecological Footprint of Consumption accounts for area required by consumption, while the Ecological Footprint of Production accounts only for the area required for production in the country only. They are related by the following equation:

<b>EF Consumption = EF Production + EF Imports - EF Exports</b>

The Ecological Footprint of Production and trade are amounts of global hectares that are calculated at the commodity level for each country in each year using enormous datasets on production and trade. The commodity-level values are then summed up and fed into the above equation for each landtype (Crop Land, Grazing Land, etc).



#### 2) WORLD HAPPINESS

In [4]:
hap_15 = pd.read_csv("wh_2015.csv")
hap_15.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [5]:
hap_16 = pd.read_csv("wh_2016.csv")
hap_16.head()

,Country,Region,Happiness Rank,Happiness Score,Lower Confidence Interval,Upper Confidence Interval,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Denmark,Western Europe,1,7.526,7.460,7.592,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939
1,Switzerland,Western Europe,2,7.509,7.428,7.590,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463
2,Iceland,Western Europe,3,7.501,7.333,7.669,1.42666,1.18326,0.86733,0.56624,0.14975,0.47678,2.83137
3,Norway,Western Europe,4,7.498,7.421,7.575,1.57744,1.12690,0.79579,0.59609,0.35776,0.37895,2.66465
4,Finland,Western Europe,5,7.413,7.351,7.475,1.40598,1.13464,0.81091,0.57104,0.41004,0.25492,2.82596


In [6]:
hap_17 = pd.read_csv("wh_2017.csv")
hap_17.head()

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


<b>DESCRIPTION OF COLUMNS:</b>
    
-<b>Country:</b> Name of the country.
    
-<b>Region:</b> Region the country belongs to.

-<b>Happiness Rank:</b> Rank of the country based on the Happiness Score.

-<b>Happiness Score:</b> A metric measured in 2015 by asking the sampled people the question: "How would you rate your happiness on a scale of 0 to 10 where 10 is the happiest."

-<b>Standard Error:</b> The standard error of the happiness score.

-<b>Economy (GDP per Capita):</b> The extent to which GDP contributes to the calculation of the Happiness Score.

-<b>Family:</b> The extent to which Family contributes to the calculation of the Happiness Score

-<b>Health (Life Expectancy):</b> The extent to which Life expectancy contributed to the calculation of the Happiness Score

-<b>Freedom:</b> The extent to which Freedom contributed to the calculation of the Happiness Score.

-<b>Trust (Government Corruption):</b> The extent to which Perception of Corruption contributes to Happiness Score.

-<b>Generosity:</b> The extent to which Generosity contributed to the calculation of the Happiness Score.

-<b>Dystopia Residual:</b> The extent to which Dystopia Residual contributed to the calculation of the Happiness Score.

<b>SUMMARY:</b>

<b>Context</b>

The World Happiness Report is a landmark survey of the state of global happiness. The first report was published in 2012, the second in 2013, the third in 2015, and the fourth in the 2016 Update. The World Happiness 2017, which ranks 155 countries by their happiness levels, was released at the United Nations at an event celebrating International Day of Happiness on March 20th. The report continues to gain global recognition as governments, organizations and civil society increasingly use happiness indicators to inform their policy-making decisions. Leading experts across fields – economics, psychology, survey analysis, national statistics, health, public policy and more – describe how measurements of well-being can be used effectively to assess the progress of nations. The reports review the state of happiness in the world today and show how the new science of happiness explains personal and national variations in happiness.

<b>Content</b>

The happiness scores and rankings use data from the Gallup World Poll. The scores are based on answers to the main life evaluation question asked in the poll. This question, known as the Cantril ladder, asks respondents to think of a ladder with the best possible life for them being a 10 and the worst possible life being a 0 and to rate their own current lives on that scale. The scores are from nationally representative samples for the years 2013-2016 and use the Gallup weights to make the estimates representative. The columns following the happiness score estimate the extent to which each of six factors – economic production, social support, life expectancy, freedom, absence of corruption, and generosity – contribute to making life evaluations higher in each country than they are in Dystopia, a hypothetical country that has values equal to the world’s lowest national averages for each of the six factors. They have no impact on the total score reported for each country, but they do explain why some countries rank higher than others.

<b>Inspiration</b>

What countries or regions rank the highest in overall happiness and each of the six factors contributing to happiness? How did country ranks or scores change between the 2015 and 2016 as well as the 2016 and 2017 reports? Did any country experience a significant increase or decrease in happiness?

<b>What is Dystopia?</b>

Dystopia is an imaginary country that has the world’s least-happy people. The purpose in establishing Dystopia is to have a benchmark against which all countries can be favorably compared (no country performs more poorly than Dystopia) in terms of each of the six key variables, thus allowing each sub-bar to be of positive width. The lowest scores observed for the six key variables, therefore, characterize Dystopia. Since life would be very unpleasant in a country with the world’s lowest incomes, lowest life expectancy, lowest generosity, most corruption, least freedom and least social support, it is referred to as “Dystopia,” in contrast to Utopia.

<b>What are the residuals?</b>

The residuals, or unexplained components, differ for each country, reflecting the extent to which the six variables either over- or under-explain average 2014-2016 life evaluations. These residuals have an average value of approximately zero over the whole set of countries. Figure 2.2 shows the average residual for each country when the equation in Table 2.1 is applied to average 2014- 2016 data for the six variables in that country. We combine these residuals with the estimate for life evaluations in Dystopia so that the combined bar will always have positive values. As can be seen in Figure 2.2, although some life evaluation residuals are quite large, occasionally exceeding one point on the scale from 0 to 10, they are always much smaller than the calculated value in Dystopia, where the average life is rated at 1.85 on the 0 to 10 scale.

<b>What do the columns succeeding the Happiness Score(like Family, Generosity, etc.) describe?</b>

The following columns: GDP per Capita, Family, Life Expectancy, Freedom, Generosity, Trust Government Corruption describe the extent to which these factors contribute in evaluating the happiness in each country. The Dystopia Residual metric actually is the Dystopia Happiness Score(1.85) + the Residual value or the unexplained value for each country as stated in the previous answer.
If you add all these factors up, you get the happiness score so it might be un-reliable to model them to predict Happiness Scores.

### CHECK DATA  TYPES, NANs AND UNIQUE VALUES

#### 1) NATIONAL FOOTPRINT ACCOUNTS

In [7]:
general_check(nfa)

{'COLNAMES': ['country',
  'ISO alpha-3 code',
  'UN_region',
  'UN_subregion',
  'year',
  'record',
  'crop_land',
  'grazing_land',
  'forest_land',
  'fishing_ground',
  'built_up_land',
  'carbon',
  'total',
  'Percapita GDP (2010 USD)',
  'population'],
 'SHAPE': (87020, 15),
 'TYPE': {'country': dtype('O'),
  'ISO alpha-3 code': dtype('O'),
  'UN_region': dtype('O'),
  'UN_subregion': dtype('O'),
  'year': dtype('int64'),
  'record': dtype('O'),
  'crop_land': dtype('float64'),
  'grazing_land': dtype('float64'),
  'forest_land': dtype('float64'),
  'fishing_ground': dtype('float64'),
  'built_up_land': dtype('float64'),
  'carbon': dtype('float64'),
  'total': dtype('float64'),
  'Percapita GDP (2010 USD)': dtype('float64'),
  'population': dtype('int64')},
 'NULLS': {'country': 0,
  'ISO alpha-3 code': 1360,
  'UN_region': 0,
  'UN_subregion': 0,
  'year': 0,
  'record': 0,
  'crop_land': 23490,
  'grazing_land': 23490,
  'forest_land': 23490,
  'fishing_ground': 23490,
  'bu

In [8]:
col_uniques(nfa)

{'COUNTRY': array(['Armenia', 'Afghanistan', 'Albania', 'Algeria', 'Angola',
        'Antigua and Barbuda', 'Argentina', 'Australia', 'Austria',
        'Bahamas', 'Bahrain', 'Barbados', 'Bangladesh', 'Bermuda',
        'Bhutan', 'Bolivia', 'Botswana', 'Brazil', 'Aruba', 'Belize',
        'Brunei Darussalam', 'Bulgaria', 'Myanmar', 'Burundi', 'Cameroon',
        'Canada', 'Cabo Verde', 'Cayman Islands',
        'Central African Republic', 'Sri Lanka', 'Chad', 'Chile',
        'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica',
        'Cuba', 'Cyprus', 'Czechoslovakia', 'Azerbaijan', 'Benin',
        'Denmark', 'Dominica', 'Dominican Republic', 'Belarus', 'Ecuador',
        'Egypt', 'El Salvador', 'Equatorial Guinea', 'Ethiopia PDR',
        'Estonia', 'Fiji', 'Finland', 'France', 'French Guiana',
        'French Polynesia', 'Djibouti', 'Georgia', 'Gabon', 'Gambia',
        'Germany', 'Bosnia and Herzegovina', 'Ghana', 'Kiribati', 'Greece',
        'Grenada', 'Guadeloupe', 'G

#### 2) WORLD HAPPINESS 

2015

In [9]:
general_check(hap_15)

{'COLNAMES': ['Country',
  'Region',
  'Happiness Rank',
  'Happiness Score',
  'Standard Error',
  'Economy (GDP per Capita)',
  'Family',
  'Health (Life Expectancy)',
  'Freedom',
  'Trust (Government Corruption)',
  'Generosity',
  'Dystopia Residual'],
 'SHAPE': (158, 12),
 'TYPE': {'Country': dtype('O'),
  'Region': dtype('O'),
  'Happiness Rank': dtype('int64'),
  'Happiness Score': dtype('float64'),
  'Standard Error': dtype('float64'),
  'Economy (GDP per Capita)': dtype('float64'),
  'Family': dtype('float64'),
  'Health (Life Expectancy)': dtype('float64'),
  'Freedom': dtype('float64'),
  'Trust (Government Corruption)': dtype('float64'),
  'Generosity': dtype('float64'),
  'Dystopia Residual': dtype('float64')},
 'NULLS': {'Country': 0,
  'Region': 0,
  'Happiness Rank': 0,
  'Happiness Score': 0,
  'Standard Error': 0,
  'Economy (GDP per Capita)': 0,
  'Family': 0,
  'Health (Life Expectancy)': 0,
  'Freedom': 0,
  'Trust (Government Corruption)': 0,
  'Generosity': 0,
 

In [10]:
col_uniques(hap_15)

{'COUNTRY': array(['Switzerland', 'Iceland', 'Denmark', 'Norway', 'Canada', 'Finland',
        'Netherlands', 'Sweden', 'New Zealand', 'Australia', 'Israel',
        'Costa Rica', 'Austria', 'Mexico', 'United States', 'Brazil',
        'Luxembourg', 'Ireland', 'Belgium', 'United Arab Emirates',
        'United Kingdom', 'Oman', 'Venezuela', 'Singapore', 'Panama',
        'Germany', 'Chile', 'Qatar', 'France', 'Argentina',
        'Czech Republic', 'Uruguay', 'Colombia', 'Thailand',
        'Saudi Arabia', 'Spain', 'Malta', 'Taiwan', 'Kuwait', 'Suriname',
        'Trinidad and Tobago', 'El Salvador', 'Guatemala', 'Uzbekistan',
        'Slovakia', 'Japan', 'South Korea', 'Ecuador', 'Bahrain', 'Italy',
        'Bolivia', 'Moldova', 'Paraguay', 'Kazakhstan', 'Slovenia',
        'Lithuania', 'Nicaragua', 'Peru', 'Belarus', 'Poland', 'Malaysia',
        'Croatia', 'Libya', 'Russia', 'Jamaica', 'North Cyprus', 'Cyprus',
        'Algeria', 'Kosovo', 'Turkmenistan', 'Mauritius', 'Hong Kong',
  

2016

In [11]:
general_check(hap_16)

{'COLNAMES': ['Country',
  'Region',
  'Happiness Rank',
  'Happiness Score',
  'Lower Confidence Interval',
  'Upper Confidence Interval',
  'Economy (GDP per Capita)',
  'Family',
  'Health (Life Expectancy)',
  'Freedom',
  'Trust (Government Corruption)',
  'Generosity',
  'Dystopia Residual'],
 'SHAPE': (157, 13),
 'TYPE': {'Country': dtype('O'),
  'Region': dtype('O'),
  'Happiness Rank': dtype('int64'),
  'Happiness Score': dtype('float64'),
  'Lower Confidence Interval': dtype('float64'),
  'Upper Confidence Interval': dtype('float64'),
  'Economy (GDP per Capita)': dtype('float64'),
  'Family': dtype('float64'),
  'Health (Life Expectancy)': dtype('float64'),
  'Freedom': dtype('float64'),
  'Trust (Government Corruption)': dtype('float64'),
  'Generosity': dtype('float64'),
  'Dystopia Residual': dtype('float64')},
 'NULLS': {'Country': 0,
  'Region': 0,
  'Happiness Rank': 0,
  'Happiness Score': 0,
  'Lower Confidence Interval': 0,
  'Upper Confidence Interval': 0,
  'Econo

In [12]:
col_uniques(hap_16)

{'COUNTRY': array(['Denmark', 'Switzerland', 'Iceland', 'Norway', 'Finland', 'Canada',
        'Netherlands', 'New Zealand', 'Australia', 'Sweden', 'Israel',
        'Austria', 'United States', 'Costa Rica', 'Puerto Rico', 'Germany',
        'Brazil', 'Belgium', 'Ireland', 'Luxembourg', 'Mexico',
        'Singapore', 'United Kingdom', 'Chile', 'Panama', 'Argentina',
        'Czech Republic', 'United Arab Emirates', 'Uruguay', 'Malta',
        'Colombia', 'France', 'Thailand', 'Saudi Arabia', 'Taiwan',
        'Qatar', 'Spain', 'Algeria', 'Guatemala', 'Suriname', 'Kuwait',
        'Bahrain', 'Trinidad and Tobago', 'Venezuela', 'Slovakia',
        'El Salvador', 'Malaysia', 'Nicaragua', 'Uzbekistan', 'Italy',
        'Ecuador', 'Belize', 'Japan', 'Kazakhstan', 'Moldova', 'Russia',
        'Poland', 'South Korea', 'Bolivia', 'Lithuania', 'Belarus',
        'North Cyprus', 'Slovenia', 'Peru', 'Turkmenistan', 'Mauritius',
        'Libya', 'Latvia', 'Cyprus', 'Paraguay', 'Romania', 'Estonia'

2017

In [13]:
general_check(hap_17)

{'COLNAMES': ['Country',
  'Happiness.Rank',
  'Happiness.Score',
  'Whisker.high',
  'Whisker.low',
  'Economy..GDP.per.Capita.',
  'Family',
  'Health..Life.Expectancy.',
  'Freedom',
  'Generosity',
  'Trust..Government.Corruption.',
  'Dystopia.Residual'],
 'SHAPE': (155, 12),
 'TYPE': {'Country': dtype('O'),
  'Happiness.Rank': dtype('int64'),
  'Happiness.Score': dtype('float64'),
  'Whisker.high': dtype('float64'),
  'Whisker.low': dtype('float64'),
  'Economy..GDP.per.Capita.': dtype('float64'),
  'Family': dtype('float64'),
  'Health..Life.Expectancy.': dtype('float64'),
  'Freedom': dtype('float64'),
  'Generosity': dtype('float64'),
  'Trust..Government.Corruption.': dtype('float64'),
  'Dystopia.Residual': dtype('float64')},
 'NULLS': {'Country': 0,
  'Happiness.Rank': 0,
  'Happiness.Score': 0,
  'Whisker.high': 0,
  'Whisker.low': 0,
  'Economy..GDP.per.Capita.': 0,
  'Family': 0,
  'Health..Life.Expectancy.': 0,
  'Freedom': 0,
  'Generosity': 0,
  'Trust..Government.Cor

In [14]:
col_uniques(hap_17)

{'COUNTRY': array(['Norway', 'Denmark', 'Iceland', 'Switzerland', 'Finland',
        'Netherlands', 'Canada', 'New Zealand', 'Sweden', 'Australia',
        'Israel', 'Costa Rica', 'Austria', 'United States', 'Ireland',
        'Germany', 'Belgium', 'Luxembourg', 'United Kingdom', 'Chile',
        'United Arab Emirates', 'Brazil', 'Czech Republic', 'Argentina',
        'Mexico', 'Singapore', 'Malta', 'Uruguay', 'Guatemala', 'Panama',
        'France', 'Thailand', 'Taiwan Province of China', 'Spain', 'Qatar',
        'Colombia', 'Saudi Arabia', 'Trinidad and Tobago', 'Kuwait',
        'Slovakia', 'Bahrain', 'Malaysia', 'Nicaragua', 'Ecuador',
        'El Salvador', 'Poland', 'Uzbekistan', 'Italy', 'Russia', 'Belize',
        'Japan', 'Lithuania', 'Algeria', 'Latvia', 'South Korea',
        'Moldova', 'Romania', 'Bolivia', 'Turkmenistan', 'Kazakhstan',
        'North Cyprus', 'Slovenia', 'Peru', 'Mauritius', 'Cyprus',
        'Estonia', 'Belarus', 'Libya', 'Turkey', 'Paraguay',
        'H

### DROP/MODIFY COLUMNS & ROWS 

#### 1) NATIONAL FOOTPRINT ACCOUNTS

In [15]:
#FILTER FOR 2014 VALUES ONLY
nfa_14 = nfa[nfa["year"]==2014]

#DROP COLUMNS: 3LETTERS COUNTRY CODE & YEAR
nfa_14.drop(labels=["ISO alpha-3 code","year"],axis=1, inplace=True)
nfa_14.head()

/home/mattia/.local/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,country,UN_region,UN_subregion,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,Percapita GDP (2010 USD),population
220,Armenia,Asia,Western Asia,BiocapPerCap,3.535664e-01,0.261703,0.093546,0.015534,0.061556,0.000000e+00,7.859056e-01,3827.34,3006000
221,Armenia,Asia,Western Asia,BiocapTotGHA,1.062874e+06,786719.027500,281214.227100,46696.840570,185046.344700,0.000000e+00,2.362550e+06,3827.34,3006000
222,Armenia,Asia,Western Asia,EFConsPerCap,6.217580e-01,0.224414,0.252693,0.006466,0.061556,8.517081e-01,2.018595e+00,3827.34,3006000
223,Armenia,Asia,Western Asia,EFConsTotGHA,1.869098e+06,674623.621400,759632.434600,19436.624040,185046.344700,2.560362e+06,6.068199e+06,3827.34,3006000
224,Armenia,Asia,Western Asia,EFExportsPerCap,1.441953e-02,0.003688,0.000152,0.007168,0.000000,1.297969e-01,1.552246e-01,3827.34,3006000


In [16]:
#TRNSFORM INTO PIVOT TABLE TO AGGREGATE DIFFERENT RECORDS BY COUNTRY
piv = nfa_14.pivot_table(index="country",columns="record")
#FLATTEN AND RECONVERT TO DF WITH NEW AGGREGATED COLS
flatten = piv.to_records()
nfa_14 = pd.DataFrame(flatten)

In [17]:
#DROP NaNs BECAUSE ARE VERY SMALL COUNTRIES, SO NOT RELEVANT
nfa_14.dropna(inplace=True)
#DROP WORLD ROW
nfa_14.drop(nfa_14.loc[nfa_14["country"]=="World"].index, inplace=True)

In [18]:
# DROP REDOUNDANT COLUMNS:

#Percapita GDP (keep first only)
nfa_14.drop(labels=[c for c in general_check(nfa_14)["COLNAMES"][1:10]],axis=1, inplace=True)

general_check(nfa_14)
#Population (keep first only)
nfa_14.drop(labels=[c for c in general_check(nfa_14)["COLNAMES"][62:71]],axis=1, inplace=True)

#All PerCapita Values (they can be computed from Population figures if needed)
nfa_14.drop(labels=[c for c in general_check(nfa_14)["COLNAMES"][2:] if "PerCap" in c]
            ,axis=1, inplace=True) 

In [19]:
# MODIFY COLUMNS HEADINGS
nfa_14.columns = [re.sub("\W","", col.replace(" ","_").replace("GHA","")) for col in nfa_14.columns]

#RENAME
nfa_14.rename(columns={"Percapita_GDP_2010_USD_EFProdTot":"GDP",
                       "population_EFProdTot":"population"}, inplace=True)

#CALCULATE NEW COLUMN CONTENT = "GDP"
nfa_14["GDP"] = nfa_14["GDP"]*nfa_14["population"]

#CALCULATE NEW COLUMNS FOR NET EF of TRADE for each cathegory: (from BUILT_UP_LAND to TOT)
cat = ["built_up_land_","carbon_","crop_land_","fishing_ground_","forest_land_","grazing_land_",
      "total_"]
for c in cat:
    nfa_14[c+"EFNetradeTot"] = (nfa_14[c+'EFImportsTot'] - nfa_14[c+'EFExportsTot'])
    nfa_14.drop([c+'EFImportsTot'], axis=1, inplace=True)
    nfa_14.drop([c+'EFExportsTot'], axis=1, inplace=True)

#REORDER COLUMNS
new_order = ['country', 'GDP','population','built_up_land_BiocapTot',
             'built_up_land_EFConsTot', 'built_up_land_EFProdTot',
             'built_up_land_EFNetradeTot',
             'carbon_BiocapTot', 'carbon_EFConsTot', 'carbon_EFProdTot',
             'carbon_EFNetradeTot',
             'crop_land_BiocapTot','crop_land_EFConsTot','crop_land_EFProdTot',
             'crop_land_EFNetradeTot',
             'fishing_ground_BiocapTot', 'fishing_ground_EFConsTot','fishing_ground_EFProdTot',
             'fishing_ground_EFNetradeTot',
             'forest_land_BiocapTot','forest_land_EFConsTot','forest_land_EFProdTot',
             'forest_land_EFNetradeTot',
             'grazing_land_BiocapTot', 'grazing_land_EFConsTot','grazing_land_EFProdTot',
             'grazing_land_EFNetradeTot', 'total_BiocapTot',
             'total_EFConsTot','total_EFProdTot','total_EFNetradeTot']

nfa_14 = nfa_14[[str(col) for col in new_order]]
    
    
#SORT BY COUNTRY IN ALPHABETIC ORDER
nfa_14.sort_values(by="country",ascending=True,inplace=True)

#RESET INDEX
nfa_14.reset_index(drop=True,inplace=True)

In [20]:
nfa_14

,country,GDP,population,built_up_land_BiocapTot,built_up_land_EFConsTot,built_up_land_EFProdTot,built_up_land_EFNetradeTot,carbon_BiocapTot,carbon_EFConsTot,carbon_EFProdTot,...,forest_land_EFProdTot,forest_land_EFNetradeTot,grazing_land_BiocapTot,grazing_land_EFConsTot,grazing_land_EFProdTot,grazing_land_EFNetradeTot,total_BiocapTot,total_EFConsTot,total_EFProdTot,total_EFNetradeTot
0,Afghanistan,1.930073e+10,31628000,1.458819e+06,1.458819e+06,1.458819e+06,0.0,0.0,4.898568e+06,3.372775e+06,...,1.920868e+06,2.998685e+05,5.914984e+06,5.910276e+06,5.704672e+06,2.056032e+05,1.594269e+07,2.442466e+07,2.041903e+07,4.005625e+06
1,Albania,1.275519e+10,2890000,1.149292e+05,1.149292e+05,1.149292e+05,0.0,0.0,2.592727e+06,1.715084e+06,...,4.794544e+05,3.593386e+05,6.316040e+05,6.904705e+05,6.316040e+05,5.886648e+04,3.038634e+06,6.188639e+06,4.111402e+06,2.077236e+06
2,Algeria,1.820511e+11,38934000,1.092607e+06,1.092607e+06,1.092607e+06,0.0,0.0,5.708592e+07,4.570675e+07,...,3.350301e+06,3.389591e+06,1.067090e+07,7.134556e+06,4.894426e+06,2.240130e+06,2.108411e+07,9.534057e+07,6.323773e+07,3.210285e+07
3,Angola,9.079613e+10,24228000,1.144945e+06,1.144945e+06,1.144945e+06,0.0,0.0,1.876227e+07,6.945604e+06,...,2.511559e+06,1.593798e+05,2.978759e+07,2.324783e+06,1.899933e+06,4.248504e+05,5.548913e+07,3.776760e+07,2.080125e+07,1.696636e+07
4,Argentina,4.436911e+11,42980000,4.513279e+06,4.513279e+06,4.513279e+06,0.0,0.0,5.505947e+07,6.704603e+07,...,9.433302e+06,1.103335e+06,7.703480e+07,3.621211e+07,4.362484e+07,-7.412722e+06,2.883548e+08,1.585845e+08,2.444948e+08,-8.591026e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,"Venezuela, Bolivarian Republic of",4.207840e+11,30694000,1.073141e+06,1.073141e+06,1.073141e+06,0.0,0.0,5.572140e+07,5.563487e+07,...,2.545099e+06,9.407228e+05,1.727013e+07,2.353039e+07,1.727013e+07,6.260258e+06,8.252364e+07,1.004069e+08,8.268420e+07,1.772270e+07
154,Viet Nam,1.475395e+11,92423000,1.038427e+07,1.038427e+07,1.038427e+07,0.0,0.0,7.273122e+07,5.816822e+07,...,1.239483e+07,5.006170e+06,8.527450e+05,9.344312e+05,2.053954e+05,7.290358e+05,9.667832e+07,1.602688e+08,1.427059e+08,1.756293e+07
155,Yemen,2.883173e+10,26184000,7.719412e+05,7.719412e+05,7.719412e+05,0.0,0.0,1.290989e+07,8.079959e+06,...,1.943818e+05,8.941688e+05,2.910332e+06,3.650952e+06,2.910332e+06,7.406201e+05,1.124006e+07,2.639133e+07,1.538549e+07,1.100584e+07
156,Zambia,2.548091e+10,15721000,6.253927e+05,6.253927e+05,6.253927e+05,0.0,0.0,3.400736e+06,1.405382e+06,...,4.649558e+06,1.289191e+05,1.330687e+07,2.511538e+06,2.462653e+06,4.888498e+04,3.157768e+07,1.499747e+07,1.282532e+07,2.172157e+06



#### 2) WORLD HAPPINESS

#### 2015

In [21]:
# MODIFY COLUMNS HEADINGS
hap_15.columns = [col.replace(" ","_").lower() for col in hap_15.columns]


#RENAME
hap_15.rename(columns={"economy_(gdp_per_capita)":"economy",
                       "health_(life_expectancy)":"health",
                       "trust_(government_corruption)":"trust",
                      "dystopia_residual":"residual"}, inplace=True)

#CALCULATE NEW COLUMN = "TRUE RESIDUAL"
hap_15["true_residual"] = hap_15["happiness_score"] - hap_15[[col for col in hap_15.columns[5:12]]].sum(axis=1)

#SORT BY COUNTRY IN ALPHABETIC ORDER
hap_15.sort_values(by="country",ascending=True,inplace=True)

#RESET INDEX
hap_15.reset_index(drop=True,inplace=True)

In [22]:
hap_15

,country,region,happiness_rank,happiness_score,standard_error,economy,family,health,freedom,trust,generosity,residual,true_residual
0,Afghanistan,Southern Asia,153,3.575,0.03084,0.31982,0.30285,0.30335,0.23414,0.09719,0.36510,1.95210,0.00045
1,Albania,Central and Eastern Europe,95,4.959,0.05013,0.87867,0.80434,0.81325,0.35733,0.06413,0.14272,1.89894,-0.00038
2,Algeria,Middle East and Northern Africa,68,5.605,0.05099,0.93929,1.07772,0.61766,0.28579,0.17383,0.07822,2.43209,0.00040
3,Angola,Sub-Saharan Africa,137,4.033,0.04758,0.75778,0.86040,0.16683,0.10384,0.07122,0.12344,1.94939,0.00010
4,Argentina,Latin America and Caribbean,30,6.574,0.04612,1.05351,1.24823,0.78723,0.44974,0.08484,0.11451,2.83600,-0.00006
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,Venezuela,Latin America and Caribbean,23,6.810,0.06476,1.04424,1.25596,0.72052,0.42908,0.11069,0.05841,3.19131,-0.00021
154,Vietnam,Southeastern Asia,75,5.360,0.03107,0.63216,0.91226,0.74676,0.59444,0.10441,0.16860,2.20173,-0.00036
155,Yemen,Middle East and Northern Africa,136,4.077,0.04367,0.54649,0.68093,0.40064,0.35571,0.07854,0.09131,1.92313,0.00025
156,Zambia,Sub-Saharan Africa,85,5.129,0.06988,0.47038,0.91612,0.29924,0.48827,0.12468,0.19591,2.63430,0.00010


### MERGE SETS

In [23]:
#UNIFORM THE NAMES OF THE COUNTRIES BEFORE MARGE

#Names in nfa, but not in hap
change_1 = [c for c in nfa_14["country"] if c not in [c for c in hap_15["country"]]]
print(change_1)

['Bahamas', 'Barbados', 'Belize', 'Brunei Darussalam', 'Congo', 'Congo, Democratic Republic of', 'Cuba', "Côte d'Ivoire", 'Equatorial Guinea', 'Fiji', 'French Guiana', 'Gambia', 'Guinea-Bissau', 'Guyana', 'Iran, Islamic Republic of', 'Korea, Republic of', "Lao People's Democratic Republic", 'Macedonia TFYR', 'Namibia', 'Papua New Guinea', 'Russian Federation', 'Saint Lucia', 'South Sudan', 'Tanzania, United Republic of', 'Timor-Leste', 'United States of America', 'Venezuela, Bolivarian Republic of', 'Viet Nam']


In [24]:
#names in hao, but not in nfa
change_2 = [c for c in hap_15["country"] if c not in [c for c in nfa_14["country"]]]
change_2

['Cambodia',
 'Comoros',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Cyprus',
 'Egypt',
 'Honduras',
 'Hong Kong',
 'Iceland',
 'Iran',
 'Ivory Coast',
 'Kosovo',
 'Laos',
 'Libya',
 'Macedonia',
 'Mauritius',
 'New Zealand',
 'North Cyprus',
 'Palestinian Territories',
 'Russia',
 'Somaliland region',
 'South Korea',
 'Syria',
 'Taiwan',
 'Tanzania',
 'United States',
 'Venezuela',
 'Vietnam']

In [25]:
# CHANGE FOR SHORTEST NAMES IN BOTH SETS
nfa_14 = nfa_14.replace({"Congo, Democratic Republic of":"Congo DR",
                         "Côte d'Ivoire":"Ivory Coast",
                         "Iran, Islamic Republic of":"Iran",
                         "Korea, Republic of":"South Korea",
                         "Lao People's Democratic Republic":"Laos",
                         "Russian Federation":"Russia",
                         "Macedonia TFYR":"Macedonia",
                         "Tanzania, United Republic of":"Tanzania",
                         "United States of America": "United States",
                         "Venezuela, Bolivarian Republic of":"Venezuela",
                         "Viet Nam":"Vietnam"})

hap_15 = hap_15.replace({"Congo (Brazzaville)":"Congo",
                        "Congo (Kinshasa)":"Congo DR"})

In [26]:
merge = nfa_14.merge(hap_15)
general_check(merge)

{'COLNAMES': ['country',
  'GDP',
  'population',
  'built_up_land_BiocapTot',
  'built_up_land_EFConsTot',
  'built_up_land_EFProdTot',
  'built_up_land_EFNetradeTot',
  'carbon_BiocapTot',
  'carbon_EFConsTot',
  'carbon_EFProdTot',
  'carbon_EFNetradeTot',
  'crop_land_BiocapTot',
  'crop_land_EFConsTot',
  'crop_land_EFProdTot',
  'crop_land_EFNetradeTot',
  'fishing_ground_BiocapTot',
  'fishing_ground_EFConsTot',
  'fishing_ground_EFProdTot',
  'fishing_ground_EFNetradeTot',
  'forest_land_BiocapTot',
  'forest_land_EFConsTot',
  'forest_land_EFProdTot',
  'forest_land_EFNetradeTot',
  'grazing_land_BiocapTot',
  'grazing_land_EFConsTot',
  'grazing_land_EFProdTot',
  'grazing_land_EFNetradeTot',
  'total_BiocapTot',
  'total_EFConsTot',
  'total_EFProdTot',
  'total_EFNetradeTot',
  'region',
  'happiness_rank',
  'happiness_score',
  'standard_error',
  'economy',
  'family',
  'health',
  'freedom',
  'trust',
  'generosity',
  'residual',
  'true_residual'],
 'SHAPE': (142, 4

In [27]:
#REORDER COLUMNS AFTER MERGE
new_order = ['country','region','population','GDP','happiness_rank',
             'happiness_score','standard_error','economy','family','health','freedom',
             'trust','generosity','residual','true_residual',             
             'built_up_land_BiocapTot','built_up_land_EFConsTot','built_up_land_EFProdTot',
             'built_up_land_EFNetradeTot','carbon_BiocapTot','carbon_EFConsTot',
             'carbon_EFProdTot','carbon_EFNetradeTot','crop_land_BiocapTot',
             'crop_land_EFConsTot','crop_land_EFProdTot','crop_land_EFNetradeTot',
             'fishing_ground_BiocapTot','fishing_ground_EFConsTot','fishing_ground_EFProdTot',
             'fishing_ground_EFNetradeTot','forest_land_BiocapTot','forest_land_EFConsTot',
             'forest_land_EFProdTot','forest_land_EFNetradeTot','grazing_land_BiocapTot',
             'grazing_land_EFConsTot','grazing_land_EFProdTot','grazing_land_EFNetradeTot',
             'total_BiocapTot','total_EFConsTot','total_EFProdTot','total_EFNetradeTot']

merge = merge[[str(col) for col in new_order]]

In [28]:
merge.head()

,country,region,population,GDP,happiness_rank,happiness_score,standard_error,economy,family,health,...,forest_land_EFProdTot,forest_land_EFNetradeTot,grazing_land_BiocapTot,grazing_land_EFConsTot,grazing_land_EFProdTot,grazing_land_EFNetradeTot,total_BiocapTot,total_EFConsTot,total_EFProdTot,total_EFNetradeTot
0,Afghanistan,Southern Asia,31628000,1.930073e+10,153,3.575,0.03084,0.31982,0.30285,0.30335,...,1.920868e+06,2.998685e+05,5.914984e+06,5.910276e+06,5.704672e+06,2.056032e+05,1.594269e+07,2.442466e+07,2.041903e+07,4.005625e+06
1,Albania,Central and Eastern Europe,2890000,1.275519e+10,95,4.959,0.05013,0.87867,0.80434,0.81325,...,4.794544e+05,3.593386e+05,6.316040e+05,6.904705e+05,6.316040e+05,5.886648e+04,3.038634e+06,6.188639e+06,4.111402e+06,2.077236e+06
2,Algeria,Middle East and Northern Africa,38934000,1.820511e+11,68,5.605,0.05099,0.93929,1.07772,0.61766,...,3.350301e+06,3.389591e+06,1.067090e+07,7.134556e+06,4.894426e+06,2.240130e+06,2.108411e+07,9.534057e+07,6.323773e+07,3.210285e+07
3,Angola,Sub-Saharan Africa,24228000,9.079613e+10,137,4.033,0.04758,0.75778,0.86040,0.16683,...,2.511559e+06,1.593798e+05,2.978759e+07,2.324783e+06,1.899933e+06,4.248504e+05,5.548913e+07,3.776760e+07,2.080125e+07,1.696636e+07
4,Argentina,Latin America and Caribbean,42980000,4.436911e+11,30,6.574,0.04612,1.05351,1.24823,0.78723,...,9.433302e+06,1.103335e+06,7.703480e+07,3.621211e+07,4.362484e+07,-7.412722e+06,2.883548e+08,1.585845e+08,2.444948e+08,-8.591026e+07


In [29]:
#COMPUTE VALUES OF GDP, POP, EFA columns in MILLIONS
big_cols = ['population','GDP','built_up_land_BiocapTot','built_up_land_EFConsTot','built_up_land_EFProdTot',
             'built_up_land_EFNetradeTot','carbon_BiocapTot','carbon_EFConsTot',
             'carbon_EFProdTot','carbon_EFNetradeTot','crop_land_BiocapTot',
             'crop_land_EFConsTot','crop_land_EFProdTot','crop_land_EFNetradeTot',
             'fishing_ground_BiocapTot','fishing_ground_EFConsTot','fishing_ground_EFProdTot',
             'fishing_ground_EFNetradeTot','forest_land_BiocapTot','forest_land_EFConsTot',
             'forest_land_EFProdTot','forest_land_EFNetradeTot','grazing_land_BiocapTot',
             'grazing_land_EFConsTot','grazing_land_EFProdTot','grazing_land_EFNetradeTot',
             'total_BiocapTot','total_EFConsTot','total_EFProdTot','total_EFNetradeTot']

for col in big_cols:
    merge[col] = merge[col]/(10**6)
merge.head()

,country,region,population,GDP,happiness_rank,happiness_score,standard_error,economy,family,health,...,forest_land_EFProdTot,forest_land_EFNetradeTot,grazing_land_BiocapTot,grazing_land_EFConsTot,grazing_land_EFProdTot,grazing_land_EFNetradeTot,total_BiocapTot,total_EFConsTot,total_EFProdTot,total_EFNetradeTot
0,Afghanistan,Southern Asia,31.628,19300.733976,153,3.575,0.03084,0.31982,0.30285,0.30335,...,1.920868,0.299869,5.914984,5.910276,5.704672,0.205603,15.942686,24.424655,20.419030,4.005625
1,Albania,Central and Eastern Europe,2.890,12755.188400,95,4.959,0.05013,0.87867,0.80434,0.81325,...,0.479454,0.359339,0.631604,0.690471,0.631604,0.058866,3.038634,6.188639,4.111402,2.077236
2,Algeria,Middle East and Northern Africa,38.934,182051.101260,68,5.605,0.05099,0.93929,1.07772,0.61766,...,3.350301,3.389591,10.670903,7.134556,4.894426,2.240130,21.084114,95.340572,63.237726,32.102846
3,Angola,Sub-Saharan Africa,24.228,90796.125960,137,4.033,0.04758,0.75778,0.86040,0.16683,...,2.511559,0.159380,29.787594,2.324783,1.899933,0.424850,55.489126,37.767605,20.801248,16.966357
4,Argentina,Latin America and Caribbean,42.980,443691.136000,30,6.574,0.04612,1.05351,1.24823,0.78723,...,9.433302,1.103335,77.034804,36.212114,43.624836,-7.412722,288.354829,158.584521,244.494778,-85.910257


### SAVE MERGED FILE

In [30]:
merge.to_csv("NFA-and-WHI-(2014-2015).csv")